In [1]:
import os
import glob
import PyPDF2
import nltk
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import itertools

# ---------- 0.1 IMPORTAR LIBRERÍAS PARA TKINTER----------
import tkinter as tk
from tkinter import filedialog, messagebox

In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ricardo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

In [5]:
#stop_words

In [6]:
def load_pdfs(folder_path):
    texts = []
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf"))

    log(f"Se han encontrado {len(pdf_files)} PDF.")
    if len(pdf_files) == 0:
        return []

    for file in pdf_files:
        try:
            with open(file, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                pages_text = []
                for page in reader.pages:
                    t = page.extract_text()
                    if t:
                        pages_text.append(t)
                texts.append("\n".join(pages_text))
                log(f"  ✔ Leído: {os.path.basename(file)}")
        except Exception as e:
            log(f"  ❌ Error leyendo {file}: {e}")

    return texts

In [7]:
def preprocess(text):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_
        for token in doc
        if token.is_alpha
        and token.lemma_ not in stop_words
        and len(token.lemma_) > 2
    ]
    return " ".join(tokens)

In [8]:
def get_cooccurrence_matrix(texts, tf_df, top_n=50):
    top_words = tf_df["word"].head(top_n).tolist()
    matrix = pd.DataFrame(0, index=top_words, columns=top_words)

    for text in texts:
        tokens = text.split()
        unique_tokens = set(tokens)
        filtered = [t for t in unique_tokens if t in top_words]

        for w1, w2 in itertools.combinations(filtered, 2):
            matrix.loc[w1, w2] += 1
            matrix.loc[w2, w1] += 1

    return matrix

In [10]:
#TKINTER UI

def log(msg):
    text_log.insert(tk.END, msg + "\n")
    text_log.see(tk.END)


def select_folder():
    folder = filedialog.askdirectory()
    if folder:
        folder_path_var.set(folder)


def run_processing():
    folder = folder_path_var.get()
    if not folder or not os.path.isdir(folder):
        messagebox.showerror("Error", "Debes seleccionar una carpeta válida.")
        return

    log("\n==============================")
    log("   INICIANDO PROCESAMIENTO")
    log("==============================\n")

    # 1. Cargar PDFs
    log("📄 Cargando PDFs...")
    raw_corpus = load_pdfs(folder)
    if len(raw_corpus) == 0:
        messagebox.showerror("Error", "No se encontraron PDFs.")
        return

    # 2. Preprocesar
    log("\n🔧 Preprocesando textos...")
    clean_corpus = [preprocess(t) for t in raw_corpus]

    # 3. TF
    if var_tf.get():
        log("\n📊 Calculando TF...")
        vectorizer_tf = CountVectorizer()
        X_tf = vectorizer_tf.fit_transform(clean_corpus)
        tf_counts = X_tf.toarray().sum(axis=0)

        tf_df = pd.DataFrame({
            "word": vectorizer_tf.get_feature_names_out(),
            "tf": tf_counts
        }).sort_values("tf", ascending=False)

        tf_df.to_csv("tf_keywords.csv", index=False)
        log("  ✔ Guardado: tf_keywords.csv")
    else:
        tf_df = None

    # 4. TF-IDF
    if var_tfidf.get():
        log("\n📈 Calculando TF-IDF...")
        vectorizer_tfidf = TfidfVectorizer(max_df=0.85, min_df=2)
        X_tfidf = vectorizer_tfidf.fit_transform(clean_corpus)

        tfidf_df = pd.DataFrame({
            "word": vectorizer_tfidf.get_feature_names_out(),
            "tfidf": X_tfidf.sum(axis=0).A1
        }).sort_values("tfidf", ascending=False)

        tfidf_df.to_csv("tfidf_keywords.csv", index=False)
        log("  ✔ Guardado: tfidf_keywords.csv")

    # 5. N-grams
    if var_ngrams.get():
        log("\n🔠 Extrayendo n-grams...")
        vectorizer_ng = CountVectorizer(ngram_range=(2, 3), min_df=2)
        X_ng = vectorizer_ng.fit_transform(clean_corpus)
        counts = X_ng.toarray().sum(axis=0)

        ng_df = pd.DataFrame({
            "ngram": vectorizer_ng.get_feature_names_out(),
            "freq": counts
        }).sort_values("freq", ascending=False)

        ng_df.to_csv("ngrams_keywords.csv", index=False)
        log("  ✔ Guardado: ngrams_keywords.csv")

    # 6. Coocurrencias
    if var_cooc.get():
        if tf_df is None:
            messagebox.showwarning(
                "Aviso",
                "Para coocurrencias es necesario activar TF.\nOmitiendo."
            )
        else:
            log("\n🔗 Calculando coocurrencias...")
            cooc = get_cooccurrence_matrix(clean_corpus, tf_df, top_n=50)
            cooc.to_csv("cooccurrence_matrix.csv")
            log("  ✔ Guardado: cooccurrence_matrix.csv")

    log("\n🎉 PROCESO COMPLETADO\n")
    messagebox.showinfo("Finalizado", "El procesamiento ha terminado correctamente.")


In [11]:

root = tk.Tk()
root.title("Extractor de Palabras Clave desde PDF")
root.geometry("700x550")

# Carpeta
frame_top = tk.Frame(root)
frame_top.pack(pady=10)

tk.Label(frame_top, text="Carpeta de PDFs:").grid(row=0, column=0, padx=5)
folder_path_var = tk.StringVar()
tk.Entry(frame_top, textvariable=folder_path_var, width=50).grid(row=0, column=1)
tk.Button(frame_top, text="Seleccionar", command=select_folder).grid(row=0, column=2, padx=5)

# Opciones
frame_opts = tk.LabelFrame(root, text="Opciones a ejecutar", padx=10, pady=10)
frame_opts.pack(pady=10)

var_tf = tk.BooleanVar(value=True)
var_tfidf = tk.BooleanVar(value=True)
var_ngrams = tk.BooleanVar(value=True)
var_cooc = tk.BooleanVar(value=True)

tk.Checkbutton(frame_opts, text="Frecuencias (TF)", variable=var_tf).pack(anchor="w")
tk.Checkbutton(frame_opts, text="TF-IDF", variable=var_tfidf).pack(anchor="w")
tk.Checkbutton(frame_opts, text="N-grams", variable=var_ngrams).pack(anchor="w")
tk.Checkbutton(frame_opts, text="Coocurrencias", variable=var_cooc).pack(anchor="w")

# Botón ejecutar
tk.Button(root, text="Ejecutar", command=run_processing, bg="#4CAF50", fg="white", height=2).pack(pady=10)

# Log
text_log = tk.Text(root, height=15, width=90)
text_log.pack(pady=10)

root.mainloop()